In [1]:
%load_ext autoreload
%autoreload 2

In [2]:


import numpy as np
import matplotlib.pyplot as plt
from space_env import Satellite, Propagator

>>> [SUCCESS] GMAT LOADED


In [3]:
# Instanciamos el satelite (esto crea el objeto Spacecraft en GMAT)
sat = Satellite("Saocom")

# Configuramos una orbita circular a 700 km de altura (LEO)
# SMA = 6371 (Radio Tierra) + 700 = 7071 km
sat.set_keplerian(
    sma=7071, 
    ecc=0.0001, 
    inc=98.2, 
    raan=0, 
    aop=0, 
    ta=0
)

print(f"Satelite '{sat.name}' configurado.")

Satelite 'Saocom' configurado.


In [4]:
print("--- Diagnóstico de Estado Inicial ---")
print(f"X: {sat.gmat_obj.GetField('X')}")
print(f"Y: {sat.gmat_obj.GetField('Y')}")
print(f"Z: {sat.gmat_obj.GetField('Z')}")
print(f"VY: {sat.gmat_obj.GetField('VY')}") # Si esto es 0.0, el satélite no tiene impulso lateral
print(f"VZ: {sat.gmat_obj.GetField('VZ')}") # Si esto es 0.0, no hay movimiento polar

--- Diagnóstico de Estado Inicial ---
X: 7070.292899999999
Y: 0
Z: 0
VY: -1.070975495268192
VZ: 7.432055064576227


In [5]:
# 1. Definir la fidelidad de la física
config_leo = {
    'gravity': 'Earth',
    'degree': 2,    # Activa J2
    'order': 2,
    'drag': True    # Resistencia atmosférica (importante en LEO)
}

# 2. Instanciar el propagador (esto crea el ForceModel y el Propagator en GMAT)
prop = Propagator("MyPropagator", config=config_leo)

In [6]:
# 5 horas en segundos
duracion_sim = 5 * 3600 

print("Propagando...")
# Esto devuelve un array de numpy: [tiempo, x, y, z]
trayectoria = prop.run(sat, duration_sec=duracion_sim, step_size=60)

print(f"¡Listo! Se generaron {trayectoria.shape[0]} puntos de datos.")

Propagando...
¡Listo! Se generaron 300 puntos de datos.


In [7]:
print (trayectoria[0],trayectoria[150])

[  60.     7070.2929    0.        0.    ] [9060.     7070.2929    0.        0.    ]


In [8]:
from astropy import units as u
from astropy.coordinates import CartesianRepresentation, GCRS, ITRS, EarthLocation
from astropy.time import Time
import plotly.graph_objects as go

def plot_ground_track(trayectoria, epoch="2000-01-01T12:00:00"):
    # 1. Extraer datos del array de GMAT
    tiempos_seg = trayectoria[:, 0]
    pos_km = trayectoria[:, 1:4]
    
    # 2. Configurar tiempos con Astropy
    t_ref = Time(epoch, format='isot', scale='utc')
    tiempos = t_ref + tiempos_seg * u.second
    
    # 3. Transformación de Inercial (GCRS) a Terrestre (ITRS)
    # GMAT usa MJ2000, que en Astropy mapeamos a GCRS con alta precisión
    cartesianas = CartesianRepresentation(pos_km.T * u.km)
    gcrs_coords = GCRS(cartesianas, obstime=tiempos)
    itrs_coords = gcrs_coords.transform_to(ITRS(obstime=tiempos))
    
    # 4. Obtener Latitud y Longitud Geodésica (WGS84)
    location = EarthLocation.from_geocentric(itrs_coords.x, itrs_coords.y, itrs_coords.z)
    lat = location.lat.value
    lon = location.lon.value
    
    # 5. Crear el Globo 3D con Plotly
    fig = go.Figure(data=go.Scattergeo(
        lat=lat,
        lon=lon,
        mode='lines',
        line=dict(width=2, color='magenta')
    ))

    fig.update_layout(
        title="Visualización Satelital Interactiva",
        geo=dict(
            projection_type='orthographic', # Esto crea el globo 3D
            showland=True, landcolor="DarkGreen",
            showocean=True, oceancolor="LightBlue",
            showlakes=True, lakecolor="Blue",
            showcountries=True
        ),
        template="plotly_dark"
    )
    return fig

# Llamada final:
figura = plot_ground_track(trayectoria)
figura.show()

In [9]:
import plotly.graph_objects as go
from astropy import units as u
from astropy.coordinates import CartesianRepresentation, GCRS, ITRS, EarthLocation
from astropy.time import Time

def graficar_2d_plotly(trayectoria, epoch="2025-01-01T12:00:00"):
    # 1. Procesamiento de coordenadas con Astropy
    t_ref = Time(epoch, format='isot', scale='utc')
    tiempos = t_ref + trayectoria[:, 0] * u.second
    
    # Convertimos de Inercial (GCRS) a Terrestre (ITRS)
    cartesianas = CartesianRepresentation(trayectoria[:, 1:4].T * u.km)
    gcrs_coords = GCRS(cartesianas, obstime=tiempos)
    itrs_coords = gcrs_coords.transform_to(ITRS(obstime=tiempos))
    
    # Obtenemos Latitud, Longitud y Altura Geodésica
    location = EarthLocation.from_geocentric(itrs_coords.x, itrs_coords.y, itrs_coords.z)
    lats = location.lat.value
    lons = location.lon.value
    alts = location.height.to(u.km).value

    # 2. Creación del gráfico 2D
    fig = go.Figure()

    # Añadir la línea de la trayectoria
    fig.add_trace(go.Scattergeo(
        lat=lats,
        lon=lons,
        mode='lines',
        line=dict(width=2, color='red'),
        name='Trayectoria GMAT',
        hoverinfo='text',
        text=[f"T: {t:.1f}s<br>Alt: {a:.1f} km" for t, a in zip(trayectoria[:, 0], alts)]
    ))

    # Configuración del Layout para Mapa 2D
    fig.update_layout(
        title="Ground Track 2D (Proyección Equirrectangular)",
        geo=dict(
            projection_type='equirectangular', # Esta es la clave para el 2D
            showland=True, landcolor="LightGreen",
            showocean=True, oceancolor="LightBlue",
            showlakes=True, lakecolor="Blue",
            showcountries=True,
            resolution=50, # Calidad de las fronteras
            lonaxis=dict(range=[-180, 180]),
            lataxis=dict(range=[-90, 90])
        ),
        margin={"r":0,"t":50,"l":0,"b":0},
        template="plotly_white"
    )
    
    return fig

# Ejecución
fig_2d = graficar_2d_plotly(trayectoria)
fig_2d.show()